## Step 0: Load ObsPy Packages

Here we are importing ObsPy modules to handle seismic data streams and UTCDateTime objects.
We will print a statement to screen to make sure this is working

In [ ]:
from obspy import Stream, UTCDateTime
print("Hello World, I ran this cell!")

Now we also import the Client Function to make Data Requests. We initialize the client object and here are setting it to pull data from IRIS

In [ ]:
from obspy.clients.fdsn import Client
client = Client("IRIS")
print("Client Initialized")

# Step 1: Find Earthquakes

We will search for events of Magnitude 7 or larger that have happened this year. These Earthquakes are detected by agencies across the glove and characteristics of the earthquake (Magnitude, location, time, slip-mechanism) are cataloged by the [U.S. Geological Survey National Earthquake Information Center](https://www.usgs.gov/programs/earthquake-hazards/national-earthquake-information-center-neic). The catalog gives you the time origin time of each event, the Epicenter location, and the size of the Earthquake.  

**Question: Typically, we expect there to be about 10 M7 and greater earthquakes each year. How many M7 or larger events do you expect to have happened so far in 2023?** 


In [ ]:
stime = UTCDateTime("2023-01-01")
etime = UTCDateTime("2023-05-30")
My_Earthquakes = client.get_events(starttime=stime,endtime=etime,minmagnitude=7.0,catalog="NEIC PDE")
print(My_Earthquakes.__str__(print_all=True))

We can quickly plot where these earthquakes are in Obspy! 

In [ ]:
My_Earthquakes.plot()

## Step 2: Find Some Seismic Stations that Recorded Earthquakes


Now that we have some earthquakes, what do we need to see how the ground moved? Seismic Data! 

We will create an "inventory object" to find seismic stations

Let's look at the Global Seismographic Network (USGS portion has network code "IU") to see which stations are available. To simplify, we will just look at stations that end in the leter "O." Below we use the wild card * to indicate that we don't care what the station name starts with as long as it ends on "O." Many of these stations were installed as part of a borehole seismometer network called the Seismic Research Observatory in the 1970s. 

In [ ]:
My_stations = client.get_stations(network="IU", station="*O",starttime=stime,endtime=etime,level='response')
My_stations.plot()

Our SNCL puzzle for seismic data is now halfway complete. We have network (IU) and stations (for instance NWAO). What locations and channels of data can we get from this station?

Let's say we want vertical component ground motion sampled at 1 sample per second. We can find this by searching for channgels that have the code "L?Z"

    Where L indicates 1 sample per second 
    ? is a wild card (we will accept any letter in this spot)
    Z indicates vertical component 

you can learn more about channel codes [here](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/)


2 options (let's do both to see if they agree!): 

1) manually search the [IRIS MDA](http://ds.iris.edu/mda/IU/NWAO/)
2) Get the information from the "inventory object" we created (next cell) for this station and looking for sensors that record "L?Z" data




In [ ]:
station_info = My_stations.select(station="NWAO", channel="L?Z")
print(station_info)

We have 3 sensors that match our request:
1. 00 LHZ
2. 10 LHZ
3. 20 LNZ

**Question: Which of these channels is different from the others? Think about why (hint read the channel code docs above or look at the MDA again).** 

We have data from 3 sensors, so we will have 3 location codes. Let's write a happy little "for loop" to get a little more information on each of our sensors. 

In [ ]:
for channel in station_info[0][0]:
    print(channel)
    

**Question: Which sensor above is the deepest? How Deep?**

**Question: What type of sensor is the 20 location code? Is this what you would expect for "LNZ" data?**


# Step 3: Request Seismic Data and Plot Seismograms 

We will get 2 days of data from May 19th to May 21st on station NWAO. 

The first step to get our data is to identify which SNCLs we want. We will specify the Network (IU), station name (NWAO), the 3 sensor locations (00,10,20), and the channels ("L?Z") we want to look at (as determined above).

We will also initilize a stream object to store the seisimic data. 

Then finally, for each seismic instrument, we will request data from IRIS and print out what we got.

 **Question: Before you run this cell below, how many different seismograms (called a "Trace" in ObsPy Speak) do you expect to receive?**
 

In [ ]:
start_of_data = UTCDateTime("2023-05-19T00:00:00")
end_of_data = UTCDateTime("2023-05-21T00:00:00")

net = "IU"
sta = "NWAO"
locs = ["00", "10", "20"]
chan = "L?Z"


My_data = Stream()
for loc in locs:
    My_data += client.get_waveforms(network=net, station = sta,
    location = loc, channel = chan, starttime = start_of_data, endtime = end_of_data) 
    
print(My_data)

Now let's plot the data! 

In [ ]:
My_data.plot(equal_scale=False)

Note that the scale of the plots are different. **Question: What are the units of the y-axis?** 

**Question: We have large signals around 03:00 on May 19th and 02:00 on May 20th on both the "00" and "10" sensors. What are these signals? (Hint: you already plotted the location of the sources). Are they really bigger on the "10" seismomter than the "00"?**

**Something to think about: However, the signal is barely visibile on the "20" - why might this be. What's different about this sensor?** 

To compare apples to apples, let's look at ground velocity on all the sensors. To do this we will remove the instrument response. 

In [ ]:
My_data.remove_response(inventory=My_stations, output='VEL')

My_data.plot(equal_scale=False)

**Question: What are the units on the Y-axis now?** 

**Something to think about: We have a really large signal that peaks about twice a day on the "00" sensor - Any idea what that is? In contrast - we have even larger signals that peak once a day (around 1 to 5 pm local time - keep in mind the time on the X-axis is UTC!). Any idea what this is?** 





Let's multiply the data by a factor of 1000 and highpass filter out these very long signals and only focus on seismic energy below 1000 s period (oscillation of the ground that are faster than once every ~17 minutes). Recall that frequency is the inverse of period. 


In [ ]:
for trace in My_data:
    trace.data*=1000

My_data.filter("highpass", freq=1/1000.)
My_data.plot(equal_scale=False)


Hmmm.....What's that big signal at the start of "20" sensor? It's filter ring from applying the high-pass filter. We will trim out data to remove that. Let's focus only of the large signal on May 20th

In [ ]:
trim_start = UTCDateTime("2023-05-19T03:00:00")
trim_end =  UTCDateTime("2023-05-19T06:00:00")

My_trimmed_data = My_data.copy()
My_trimmed_data.trim(trim_start,trim_end)


My_trimmed_data.plot()

Alright, we finally have estimates of vertical ground velocity (in mm/s) for the M 7.7 Earthquake on May 20th for 3 different sensors. 

**Question: Does the amplitude of ground motion between the "00" and "10" agree now?**

**Question: Which sensor looks different than the others? Any idea why?**

REVIEW:

We initiated an IRIS client.

Step 1: We used the client to pull Earthquakes from the National Earthquake Informatin Center Catalog

Step 2: We used the same client to find some GSN seismic stations, and get information about them (type of instruments and data available, response information). This information is also available from the MDA

Step 3: We then used the SNCL of a seismic station to request data for an Earthquake.  We removed the response to go from digital counts to ground velocity, and filtered the data to get a better view of the Earthquake. 
